In [ ]:
import pyaudio
from numpy import zeros,linspace,short,fromstring,hstack,transpose,log2, log
from scipy import fft, signal
from time import sleep
import struct
from scipy.signal import hamming, convolve

frequencyoutput = True

NUM_SAMPLES = 2048
SAMPLING_RATE = 48000
pa = pyaudio.PyAudio()
_stream = pa.open(format=pyaudio.paInt16,
                  channels=1, rate=SAMPLING_RATE,
                  input=True,
                  frames_per_buffer=NUM_SAMPLES)

print("Detecting Frequencies. Press CTRL-C to quit.")

while True:
    while _stream.get_read_available()< NUM_SAMPLES: sleep(0.01)
    audio_data  = fromstring(_stream.read(
        _stream.get_read_available()), dtype=short)[-NUM_SAMPLES:]
    
    normalized_data = audio_data / 32768.0    
    w = hamming(2048)  
        
    intensity = abs(w*fft.fft(normalized_data))[:1024]

    
    if frequencyoutput:
        which = intensity[1:].argmax()+1
        # use quadratic interpolation around the max
        if which != len(intensity)-1:
            y0,y1,y2 = log(intensity[which-1:which+2:])
            x1 = (y2 - y0) * .5 / (2 * y1 - y2 - y0)
            # find the frequency and output it
            thefreq = (which+x1)*SAMPLING_RATE/NUM_SAMPLES
        else:
            thefreq = which*SAMPLING_RATE/NUM_SAMPLES
      
        if(audio_data[0] > 275):
            print(thefreq)
            print(audio_data)  


Detecting Frequencies. Press CTRL-C to quit.


<ipython-input-1-2d7988d361be>:22: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  audio_data  = fromstring(_stream.read(


131.62276425320928
[ 378  372  194 ... -279 -255 -313]
1001.3492620318883
[1652 1423 1163 ... -298 -109   85]
1002.4713024103712
[  293   500   699 ... -1660 -1835 -1973]
1001.4081126226222
[1580 1359 1107 ... -584 -389 -188]
76.03870042088182
[339 375 395 ...  57  92 122]
1005.0317428512984
[523 390 245 ... 311 439 561]
1003.4790671781968
[  678   800   925 ... -1024 -1075 -1116]
1004.676440636562
[648 524 397 ... -31  92 232]
234.53365448812892
[ 304  303  295 ... -205 -206 -209]
1006.2575526026195
[ 1447  1528  1571 ... -1238 -1101  -935]
1006.5332663843183
[ 1461  1500  1506 ... -1347 -1212 -1059]
1005.700560387071
[1323 1378 1397 ... -681 -602 -514]
1005.1328092471067
[1034  885  718 ... -333 -184  -43]
1005.1239302925784
[1075  930  758 ... -251 -104   37]
8204.528964802948
[398 653 379 ...  64 276 317]
6568.31190113971
[378 314 376 ... 123  26   5]
8534.712672249972
[303 378  68 ... 159  65 229]
8793.310707315239
[ 591  621   53 ...  208 -216 -364]
7289.648493306294
[ 317   26 -